In [ ]:
"""
RAMS query program

Created on Wed Jul 5 11:49:17 2017

@author: Tingting Huang

thuang1@iastate.edu

"""

# Step 1

Use your own directory and filename

Must have column name: routeId, MM


In [ ]:
dirpath = 'C:/Users/thuang1/Desktop/REST API/Weather/'
filename = 'MM_Found_sensors.csv'

# Step 2

a. Input your attributes of interest.

go to https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer to check layerId and field names

In [ ]:
layerId = 78

fields = ['SPEED_LIMIT']

b. Load Functions

In [ ]:
def get_locations(df):
    d = []
    for i in xrange(len(df)):
        x = {"routeId":df.iloc[i]['routeId'],"measure":df.iloc[i]['MM']}
        d.append(x)
    locations = json.dumps(d, ensure_ascii=False)
    
    return locations

In [ ]:
def get_attributes(df):
    
    
    locations=get_locations(df)
    
    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/queryAttributeSet?'
    data = {'f':'json','locations': locations,'attributeSet':attributes, 'outSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()

    try:
        e=r['error']
        print('Error in get attributes!')
    except:
        pass

    resall=[]
    datall=[]
    for i in r['features']:
        res=[i['attributes']['route_id'], i['attributes']['measure']]
        for j in fields:
            res.extend([i['attributes'][str(j)]])
        resall.append(res)
    datall=np.array(resall)

    cols = ['route_id','measure']
    cols.extend(fields)
    df2=pd.DataFrame(datall, columns=cols)
    
    x=df2.duplicated()
    
    if sum(x)>0:
        print ('Duplicated records found!')
    
    df2.measure=pd.to_numeric(df2.measure)
    
    df3 = pd.merge(df,df2, how='left', left_on=['routeId','MM'], right_on=['route_id','measure'])
    
    return df3

c. Use n to slice your data into chunks with n rows in each one.

The API only returns at most 1500 responses, thus, 0<=n<=1500

In [ ]:
import pandas as pd
import numpy as np
import json
import requests

# load your data

df = pd.read_csv(dirpath+filename)

n=1000

ans=[]
for i in xrange(len(df)/n+1):
    ans.append(df[i*n:i*n+n])
    
# create attribute array
attr = [{"layerId":layerId,"fields":fields}]
attributes = json.dumps(attr, ensure_ascii=False)


d. Main function

In [ ]:
from datetime import datetime
now1=datetime.now()

final=[]
i=0
for a in ans:
    final.append(get_attributes(a))
    
    i=i+1
    print('Success! Progress: '+str(round(float(i)/len(ans)*100,2))+"%")
final=pd.concat(final,axis=0).reset_index(drop=True)

now2=datetime.now()
print('Runtime: '+str(now2-now1))

e. Save the results

In [ ]:
final.to_csv(dirpath+'Speed_Limit_Found_'+filename, header=True, index=False)